In [2]:
#hide
#skip
%config Completer.use_jedi = False
# upgrade fastrl on colab
! [ -e /content ] && pip install -Uqq fastrl['dev'] pyvirtualdisplay && \
                     apt-get install -y xvfb python-opengl > /dev/null 2>&1 
# NOTE: IF YOU SEE VERSION ERRORS, IT IS SAFE TO IGNORE THEM. COLAB IS BEHIND IN SOME OF THE PACKAGE VERSIONS

In [3]:
# hide
from fastcore.imports import in_colab
# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbverbose.showdoc import *
    from nbdev.imports import *
    if not os.environ.get("IN_TEST", None):
        assert IN_NOTEBOOK
        assert not IN_COLAB
        assert IN_IPYTHON
else:
    # Virutual display is needed for colab
    from pyvirtualdisplay import Display
    display = Display(visible=0, size=(400, 300))
    display.start()

In [4]:
# default_exp memory.tensorboard

In [5]:
# export
# Python native modules
import os
from fnmatch import fnmatch
from warnings import warn
# Third party libs
from PIL import Image
from IPython.display import HTML
from io import BytesIO
import plotly.express as px
import numpy as np
import pandas as pd
from fastcore.all import *
from tensorboard.backend.event_processing import event_accumulator
# Local modules
from fastrl.core import *

# Memory Tensorboard
> Tensorboard / Plotly visualizations using tensorboard as a backend within the context of memory

In [6]:
# hide
SHOW_TENSOR_BOARD=True
if not os.environ.get("IN_TEST", None) and SHOW_TENSOR_BOARD:
    run_tensorboard(samples_per_plugin='images=2000')

Selecting TensorBoard with logdir runs (started 0:03:52 ago; port 6006, pid 1530).


In [7]:
# export
def create_event_accumulator(
        p:Optional[Path]=None, # Path or in-complete path to a tensorboard event directory. ref `pattern`
        pattern:str='*', # Will filter paths at `cwd` if `p` is None, or filter at `p`
        verbose:bool=False,
        images=None, # Reference EventAccumulator `size_guidance`
        scalars=None, # Reference EventAccumulator `size_guidance`
        audio=None, # Reference EventAccumulator `size_guidance`
        histograms=None, # Reference EventAccumulator `size_guidance`
        compressed_histograms=None, # Reference EventAccumulator `size_guidance`
    )->event_accumulator.EventAccumulator:
    "Creates an event accumulator at the first directory at `p`."
    p=ifnone(p,Path.cwd())
    events=L([o for o in p.ls() if fnmatch(str(o),pattern)]).sorted()
    
    if not events:
        raise Exception(f'No events were found at {p}'+('' if pattern is None else f' and pattern {pattern}'))
    if len(events)>1 and verbose:
        warn(f'Got {len(events)} events. Using the most recent one.')
    if verbose:
        print('Using: ',str(events[-1]))
    ea=event_accumulator.EventAccumulator(str(events[-1]),
        size_guidance={ 
            event_accumulator.COMPRESSED_HISTOGRAMS: ifnone(compressed_histograms,500),
            event_accumulator.IMAGES: ifnone(images,100),
            event_accumulator.AUDIO: ifnone(audio,4),
            event_accumulator.SCALARS: ifnone(scalars,0),
            event_accumulator.HISTOGRAMS: ifnone(histograms,1),
        })
    ea.Reload()
    return ea

In [8]:
ea=create_event_accumulator(Path('runs'),verbose=True)

Using:  runs/Nov13_19-16-44_6104b238973c_dqn_target


In [9]:
# export
def diff_tags(a,b):
    return ''.join(set(a.split('/'))-set(b.split('/')))

In [10]:
diff_tags('25/td_error','*/td_error')

'25'

In [11]:
diff_tags('experience_replay/25/td_error','experience_replay/*/td_error')

'25'

In [12]:
# export
def scalars2value(scalars:list): return list(map(lambda o:o.value,scalars))

def scalar2line(
        tag_pattern:str, # Tags that will match this pattern to be displayed.
        ea:event_accumulator.EventAccumulator, # The event accumulator to load the images
        start:int=0, # Where in the list of images to start loading
        end:Optional[int]=None, # Optionally where to stop loading images
        step:int=1 # Number of steps betwee image reads
    ):
    tags=[t for t in ea.Tags()['scalars'] if fnmatch(t,tag_pattern)]
    if not tags: warn(f'There are not tags with pattern {tag_pattern}, there are: \n{ea.Tags()["scalars"]}')
    # slice the scalars
    sliced={t:scalars2value(ea.Scalars(t)[start:end:step]) for t in tags}
    # Create the data frame
    df=pd.DataFrame(
        data={'values':np.array(list(sliced.values())).reshape(-1,),
              'tags':np.array([[diff_tags(k,tag_pattern)]*len(v) 
                               for k,v in sliced.items()]).reshape(-1,),
              'steps':np.array([np.arange(len(v)) for v in sliced.values()]).reshape(-1,)}
    )
    
    value_array=np.array(list(sliced.values()))
    return px.line(data_frame=df,x='steps',y='values',animation_frame="tags",
                   title=tag_pattern,
            range_y=[value_array.min(),value_array.max()])

In [13]:
expected_reward_line=scalar2line('experience_replay/*/expected_reward/action_dim_0',ea=ea)

/tmp/ipykernel_1571/1810058336.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  data={'values':np.array(list(sliced.values())).reshape(-1,),
/tmp/ipykernel_1571/1810058336.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  'tags':np.array([[diff_tags(k,tag_pattern)]*len(v)
/tmp/ipykernel_1571/1810058336.py:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' w

TypeError: unhashable type: 'list'

In [ ]:
td_error_line=scalar2line('experience_replay/*/td_error',ea=ea)

In [14]:
# export
def _td_gif2np(img):
    return np.array(Image.open(BytesIO(img.encoded_image_string)).convert('RGB')).copy()

def images2animation(
        tag_pattern:str, # Tags that will match this pattern to be displayed.
        ea:event_accumulator.EventAccumulator, # The event accumulator to load the images
        start:int=0, # Where in the list of images to start loading
        end:Optional[int]=None, # Optionally where to stop loading images
        step:int=1 # Number of steps betwee image reads
    ):
    tags=[t for t in ea.Tags()['images'] if fnmatch(t,tag_pattern)]
    if not tags: 
        warn(f'There are no images with pattern {tag_pattern} given tags: '+'\n'.join(ea.Tags()['images']))
        return None
    
    imgs=ea.Images(tags[-1])[start:end:step]
    if not imgs: 
        warn(f'There are no images in {tags[-1]}')
        return None
    
    np_imgs=[_td_gif2np(o) for o in imgs]
    return px.imshow(np.array(np_imgs),animation_frame=0)

In [15]:
images_animation=images2animation(
    '*',
    ea=ea
)

/tmp/ipykernel_1571/3112655825.py:14: UserWarning: There are no images with pattern * given tags: 
  warn(f'There are no images with pattern {tag_pattern} given tags: '+'\n'.join(ea.Tags()['images']))


In [16]:
# export
def figures_to_html(*figs):    
    html="<html><head></head><body>" + "\n"
    for fig in figs: html+=fig.to_html().split('<body>')[1].split('</body>')[0]
    html+="</body></html>" + "\n"
    display(HTML(html))

In [17]:
# export
@delegates(create_event_accumulator)
def create_experience_replay_fig(
        tag_pattern_expected_reward:Union[str,list]='experience_replay/*/expected_reward/action_dim_0', # Tags about expected reward
        tag_pattern_td_error:str='experience_replay/*/td_error', # Tags about td_error
        tag_pattern_images:str='*', # Tags about images. 
        start:int=0, # Where in the list of images to start loading
        end:Optional[int]=None, # Optionally where to stop loading images
        step:int=1, # Number of steps betwee image reads
        **kwargs):
    ea=create_event_accumulator(**kwargs)
    
    common_args=dict(start=start,end=end,step=step)
    if isinstance(tag_pattern_expected_reward,str):
        expected_reward_line=(scalar2line(tag_pattern_expected_reward,ea=ea,**common_args),)
    else:
        expected_reward_line=tuple(scalar2line(tper,ea=ea,**common_args) 
                                   for tper in tag_pattern_expected_reward)
    td_error_line=scalar2line(tag_pattern_td_error,ea=ea,**common_args)
    
    if tag_pattern_images is not None:
        images_animation=images2animation(tag_pattern_images,ea=ea,**common_args)
    
        return figures_to_html(
            td_error_line,
            images_animation,
            *expected_reward_line
        )
    else:
        return figures_to_html(
            td_error_line,
            *expected_reward_line
        )


In [18]:
create_experience_replay_fig(p=Path('runs'),images=0,start=900,
                             tag_pattern_images=None,
                            tag_pattern_expected_reward=[
                                                        'experience_replay/*/retrospective_action',
                                                         'experience_replay/*/expected_reward/action_dim_0',
                                                        'experience_replay/*/expected_reward/action_dim_1',
                                                        'experience_replay/*/action'])

/tmp/ipykernel_1571/1810058336.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  data={'values':np.array(list(sliced.values())).reshape(-1,),
/tmp/ipykernel_1571/1810058336.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  'tags':np.array([[diff_tags(k,tag_pattern)]*len(v)
/tmp/ipykernel_1571/1810058336.py:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' w

TypeError: unhashable type: 'list'

In [ ]:
# hide
from fastcore.imports import in_colab

# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev.export import *
    from nbdev.export2html import *
    from nbverbose.cli import *
    make_readme()
    notebook2script(silent=True)
    